In [2]:
using ResumableFunctions, StatsBase, Statistics
using FFTW, FITSIO, Intervals
using DocStringExtensions
using LinearAlgebra
using Random
using RecipesBase
using Test
using Distributions
include("events.jl")
include("lightcurve.jl")
include("gti.jl")
include("fourier.jl")
function merge_overlapping_gtis(gtis::Matrix{Float64})
    if size(gtis, 1) <= 1
        return gtis
    end

    # Sort by start time
    sort_indices = sortperm(view(gtis, :, 1))
    sorted_gtis = gtis[sort_indices, :]

    # Pre-allocate merged array
    merged = Matrix{Float64}(undef, size(gtis, 1), 2)
    merged_count = 0

    current_start = sorted_gtis[1, 1]
    current_stop = sorted_gtis[1, 2]

    for i in 2:size(sorted_gtis, 1)
        start_time = sorted_gtis[i, 1]
        stop_time = sorted_gtis[i, 2]

        # Check if intervals overlap or touch (with small tolerance)
        if start_time <= current_stop + 1e-6
            current_stop = max(current_stop, stop_time)
        else
            # No overlap, add current interval to merged
            merged_count += 1
            merged[merged_count, 1] = current_start
            merged[merged_count, 2] = current_stop
            current_start = start_time
            current_stop = stop_time
        end
    end

    # Add the final interval
    merged_count += 1
    merged[merged_count, 1] = current_start
    merged[merged_count, 2] = current_stop

    return merged[1:merged_count, :]
end

merge_overlapping_gtis (generic function with 1 method)

In [3]:
"""
Represents a power spectrum computed from time series data.

Fields:
- freq: Frequency bins
- power: Power values for each frequency
- power_err: Uncertainties on power values
- norm: Normalization type ("frac", "leahy", "abs", "none")
- df: Frequency resolution
- nphots: Total number of photons
- m: Number of segments averaged (1 for single spectrum)
- metadata: Original metadata from input data
"""
struct Powerspectrum{T}
    freq::Vector{T}          # Frequencies
    power::Vector{T}         # Power values 
    power_err::Vector{T}     # Power errors
    norm::String            # Normalization type
    df::T                   # Frequency resolution (1/(N*bin_size))
    nphots::Int            # Total number of photons
    m::Int                 # Number of segments (1 for single spectrum)
    metadata::Union{LightCurveMetadata, FITSMetadata} # Original metadata
end

"""
Represents an averaged power spectrum computed from multiple segments.

Fields:
- freq: Frequency bins
- power: Averaged power values
- power_err: Uncertainties on averaged powers
- norm: Normalization type
- df: Frequency resolution
- segment_size: Size of segments used in averaging
- nphots: Total number of photons
- m: Number of segments averaged
- mean_rate: Mean count rate
- metadata: Original metadata from input data
"""
struct AveragedPowerspectrum{T}
    freq::Vector{T}         # Frequencies
    power::Vector{T}        # Averaged power values
    power_err::Vector{T}    # Errors on averaged powers
    norm::String           # Normalization type
    df::T                  # Frequency resolution
    segment_size::T        # Size of segments
    nphots::Int           # Total photons
    m::Int                # Number of segments averaged
    mean_rate::T          # Mean count rate
    metadata::Union{LightCurveMetadata, FITSMetadata} # Original metadata
end


AveragedPowerspectrum

In [4]:
function create_segments(events::EventList, segment_duration::Real; bin_size::Real = 1.0)
    # Get actual time range from the data
    t_start = minimum(events.times)
    t_stop = maximum(events.times)
    total_time = t_stop - t_start
    println("Data time range: $(t_start) to $(t_stop) ($(total_time) seconds total)")
    
    # Calculate number of segments
    n_segments = ceil(Int, total_time / segment_duration)
    println("Creating $(n_segments) segments of $(segment_duration) seconds each") 
    
    segments = Vector{LightCurve}() 
    
    for i in 1:n_segments
        start_time = t_start + (i-1) * segment_duration
        stop_time = min(t_start + i * segment_duration, t_stop)
        
        # Filter events in this segment using matrix operations
        event_times = events.times
        mask = (event_times .>= start_time) .& (event_times .<= stop_time)
        segment_events = event_times[mask]
        
        events_in_segment = sum(mask)
        println("Segment $i: $(events_in_segment) events from $(start_time) to $(stop_time)")
        
        # Create time bins for this segment
        time_edges = collect(start_time:bin_size:stop_time)
        if length(time_edges) < 2
            time_edges = [start_time, stop_time]
        end
        
        # Bin centers
        time_centers = (time_edges[1:end-1] + time_edges[2:end]) / 2
        n_bins = length(time_centers)
        
        # Initialize counts
        counts = zeros(Int, n_bins)
        
        # Histogram events into bins using matrix operations
        if events_in_segment > 0
            # Use searchsortedlast for efficient binning
            for event_time in segment_events
                bin_idx = searchsortedlast(time_edges, event_time)
                if bin_idx > 0 && bin_idx <= n_bins
                    counts[bin_idx] += 1
                end
            end
        end
        
        # Create light curve data matrix: [time, counts, errors]
        # Calculate Poisson errors
        count_errors = sqrt.(max.(counts, 1))  # Avoid sqrt(0)
        
        # Create the data matrix
        lc_matrix = hcat(time_centers, counts, count_errors)
        
        # Create dummy metadata
        dummy_metadata = LightCurveMetadata(
            "Unknown", "Unknown", "Unknown", 0.0, 
            (start_time, stop_time), bin_size,
            Vector{Dict{String,Any}}(), Dict{String,Any}()
        )
        
        # Create LightCurve object
        lc = LightCurve(
            lc_matrix[:, 1],  # time
            bin_size,         # dt
            Int.(lc_matrix[:, 2]),  # counts
            lc_matrix[:, 3],  # count_error
            nothing,          # exposure
            Vector{EventProperty{Float64}}(),  # properties
            dummy_metadata,   # metadata
            :poisson         # err_method
        )
        
        push!(segments, lc)
    end
    
    return segments
end

create_segments (generic function with 1 method)

In [5]:
function Powerspectrum(lc::LightCurve{T}; norm::String="frac") where {T<:Real}
    bin_size = lc.metadata.bin_size
    n_bins = length(lc.counts)
    
    # Validate input
    n_bins > 1 || throw(ArgumentError("Light curve must have more than 1 bin"))
    bin_size > 0 || throw(ArgumentError("Bin size must be positive"))
    
    # Calculate FFT
    ft = fft(lc.counts)
    
    # Get frequency array in Hz
    freqs = fftfreq(n_bins, 1/bin_size)
    pos_freq_idx = positive_fft_bins(n_bins)
    freqs = freqs[pos_freq_idx]
    
    # Get frequency resolution in Hz
    df = 1 / (n_bins * bin_size) 
    
    # Calculate power
    unnorm_power = abs2.(ft[pos_freq_idx])
    
    # Normalize if requested
    power = if norm != "none"
        normalize_periodograms(
            unnorm_power,
            bin_size,
            n_bins;
            mean_flux = mean(lc.counts),
            n_ph = sum(lc.counts),
            norm = norm
        )
    else
        unnorm_power
    end
    
    # Calculate error estimates (assuming white noise)
    power_err = if norm in ["frac", "rms"]
        power ./ sqrt(1)  # For single spectrum, error = power for exponential distribution
    elseif norm == "leahy"
        fill(2.0, length(power))  # Leahy normalization has constant error of 2
    else
        sqrt.(power)  # Generic Poisson error
    end
    
    return Powerspectrum{T}(
        freqs,
        power,
        power_err,
        norm,
        df,
        sum(lc.counts),
        1,
        lc.metadata
    )
end
"""
Construct an AveragedPowerspectrum using Bartlett's method from EventList
"""
function AveragedPowerspectrum(events::EventList{TimeType,MetaType}, segment_size::Real, bin_size::Real=1.0; 
                             norm::String="frac") where {TimeType<:AbstractVector, MetaType<:FITSMetadata}
    
    # Validate inputs
    segment_size > 0 || throw(ArgumentError("Segment size must be positive"))
    bin_size > 0 || throw(ArgumentError("Bin size must be positive"))
    
    segments = create_segments(events, segment_size, bin_size=bin_size)
    if isempty(segments)
        throw(ArgumentError("No valid segments created after GTI filtering"))
    end
    
    # Get dimensions from first segment
    first_segment = segments[1]
    n_bins = length(first_segment.counts)
    
    # Get frequency array in Hz for one segment
    freqs = fftfreq(n_bins, 1/bin_size)
    pos_freq_idx = positive_fft_bins(n_bins)
    freqs = freqs[pos_freq_idx]
    
    # Get frequency resolution in Hz
    df = 1 / (n_bins * bin_size)
    
    # Initialize arrays for accumulating spectra
    T = eltype(TimeType)  # Get the element type (Float64) from the vector type
    total_power = zeros(T, length(pos_freq_idx))
    total_counts = 0
    valid_segments = 0
    
    # Process each segment with improved filtering
    for segment in segments
        # Calculate basic statistics
        segment_counts = sum(segment.counts)
        mean_flux = mean(segment.counts)
        
        # Skip segments with insufficient data or problematic mean flux
        # More stringent filtering to avoid NaN in normalization
        if segment_counts < 100 || mean_flux <= 1e-10 || isnan(mean_flux) || isinf(mean_flux)
            continue
        end
        
        # Additional check for fractional normalization
        if norm == "frac" && mean_flux <= 1e-6
            continue  # Skip segments with very low mean flux for frac normalization
        end
        
        # Calculate FFT for this segment
        ft = fft(segment.counts)
        
        # Calculate power for positive frequencies
        unnorm_power = abs2.(ft[pos_freq_idx])
        
        # Check for problematic unnormalized power
        if any(isnan.(unnorm_power)) || any(isinf.(unnorm_power))
            continue
        end
        
        # Normalize if requested
        power = if norm != "none"
            try
                normalize_periodograms(
                    unnorm_power,
                    bin_size,
                    n_bins;
                    mean_flux = mean_flux,
                    n_ph = segment_counts,
                    norm = norm
                )
            catch e
                @warn "Normalization failed for segment with mean_flux=$mean_flux, sum=$segment_counts: $e"
                continue  # Skip this segment if normalization fails
            end
        else
            unnorm_power
        end
        
        # Final check for NaN/Inf in normalized power
        if any(isnan.(power)) || any(isinf.(power))
            @warn "NaN/Inf detected in normalized power for segment with mean_flux=$mean_flux"
            continue
        end
        
        # Accumulate
        total_power .+= power
        total_counts += segment_counts
        valid_segments += 1
    end
    
    valid_segments > 0 || throw(ArgumentError("No valid segments after filtering"))
    
    # Calculate averages
    avg_power = total_power ./ valid_segments
    mean_rate = total_counts / (valid_segments * segment_size)
    
    # Calculate error estimates for averaged spectrum
    power_err = if norm in ["frac", "rms"]
        avg_power ./ sqrt(valid_segments)  # Standard error of mean
    elseif norm == "leahy"
        fill(2.0/sqrt(valid_segments), length(avg_power))  # Leahy with averaging
    else
        sqrt.(avg_power ./ valid_segments)  # Generic error propagation
    end
    
    println("Used $valid_segments out of $(length(segments)) segments for averaging")
    
    return AveragedPowerspectrum{T}(
        freqs,
        avg_power,
        power_err,
        norm,
        df,
        segment_size,
        total_counts,
        valid_segments,
        mean_rate,
        events.meta
    )
end

AveragedPowerspectrum

In [6]:
#here is simple code why i am using fftfreq like this in our powerspectrum :}

In [ ]:
bin_size = 0.1
n_bins = 1024
freqs = fftfreq(n_bins, 1/bin_size)
pos_freq_idx = positive_fft_bins(n_bins)
freqs = freqs[pos_freq_idx]

511-element Vector{Float64}:
 0.009765625
 0.01953125
 0.029296875
 0.0390625
 0.048828125
 0.05859375
 0.068359375
 0.078125
 0.087890625
 0.09765625
 ⋮
 4.912109375
 4.921875
 4.931640625
 4.94140625
 4.951171875
 4.9609375
 4.970703125
 4.98046875
 4.990234375

In [8]:
events = readevents("ni1200120104_0mpu7_cl.evt", load_gti=true, sort=true)

Found GTI data: 16 intervals
GTI time range: 1.3253976595089495e8 to 1.3261337476368216e8


EventList with 21244574 times and energies


In [13]:
lc = create_lightcurve(events, 0.1) 

[ Info: Created light curve: 731950 bins, bin size = 0.1 s


LightCurve{Float64}([1.3254001405e8, 1.3254001415e8, 1.3254001425e8, 1.3254001435e8, 1.3254001445e8, 1.3254001455e8, 1.3254001465e8, 1.3254001475e8, 1.3254001485e8, 1.3254001495e8  …  1.3261320805e8, 1.3261320815e8, 1.3261320825e8, 1.3261320835e8, 1.3261320845e8, 1.3261320855e8, 1.3261320865e8, 1.3261320875e8, 1.3261320885e8, 1.3261320895e8], 0.1, [118, 101, 112, 155, 167, 204, 253, 167, 151, 113  …  677, 451, 321, 302, 290, 341, 352, 406, 393, 500], [10.862780491200215, 10.04987562112089, 10.583005244258363, 12.449899597988733, 12.922847983320086, 14.2828568570857, 15.905973720586866, 12.922847983320086, 12.288205727444508, 10.63014581273465  …  26.019223662515376, 21.236760581595302, 17.916472867168917, 17.378147196982766, 17.029386365926403, 18.466185312619388, 18.76166303929372, 20.149441679609886, 19.82422760159901, 22.360679774997898], [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1  …  0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1], EventProperty{Float64}[EventProperty{Floa

In [14]:
ps = Powerspectrum(lc, norm="frac")

Powerspectrum{Float64}([1.3662135391761732e-5, 2.7324270783523464e-5, 4.0986406175285196e-5, 5.464854156704693e-5, 6.831067695880866e-5, 8.197281235057039e-5, 9.563494774233212e-5, 0.00010929708313409385, 0.00012295921852585557, 0.00013662135391761732  …  4.999863378646082, 4.999877040781474, 4.999890702916866, 4.999904365052258, 4.9999180271876495, 4.999931689323041, 4.999945351458432, 4.999959013593824, 4.999972675729216, 4.999986337864608], [48934.81258829991, 2511.5662314050205, 9895.930162889203, 2189.3932716059444, 4211.791914583046, 2095.01725810083, 2862.7334774865444, 3386.069952399912, 3345.5649471123324, 8343.087989548285  …  0.0893114052581714, 0.053877114582617305, 0.02745410456644777, 0.04566658579727494, 0.03884709225316456, 0.02854192232541811, 0.058004734286808024, 0.07242759638814562, 0.054088929148272465, 0.031153591974930145], [48934.81258829991, 2511.5662314050205, 9895.930162889203, 2189.3932716059444, 4211.791914583046, 2095.01725810083, 2862.7334774865444, 3386.

In [9]:
avg_ps = AveragedPowerspectrum(events, 100.0, 0.1, norm="leahy")

Data time range: 1.3254001400020401e8 to 1.3261320899981467e8 (73194.99961066246 seconds total)
Creating 732 segments of 100.0 seconds each
Segment 1: 309011 events from 1.3254001400020401e8 to 1.3254011400020401e8
Segment 2: 306443 events from 1.3254011400020401e8 to 1.3254021400020401e8
Segment 3: 311768 events from 1.3254021400020401e8 to 1.3254031400020401e8
Segment 4: 337117 events from 1.3254031400020401e8 to 1.3254041400020401e8
Segment 5: 427050 events from 1.3254041400020401e8 to 1.3254051400020401e8
Segment 6: 327922 events from 1.3254051400020401e8 to 1.3254061400020401e8
Segment 7: 266302 events from 1.3254061400020401e8 to 1.3254071400020401e8
Segment 8: 296734 events from 1.3254071400020401e8 to 1.3254081400020401e8
Segment 9: 313840 events from 1.3254081400020401e8 to 1.3254091400020401e8
Segment 10: 41060 events from 1.3254091400020401e8 to 1.3254101400020401e8
Segment 11: 0 events from 1.3254101400020401e8 to 1.3254111400020401e8
Segment 12: 0 events from 1.32541114000

AveragedPowerspectrum{Float64}([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1  …  4.9, 4.91, 4.92, 4.93, 4.94, 4.95, 4.96, 4.97, 4.98, 4.99], [28383.37927567591, 16826.114653053915, 10425.229538741172, 5674.742191932274, 3082.190917144407, 2312.3595989747446, 2144.928938038708, 2044.1469239404184, 1659.1807288100154, 1365.8298327657333  …  11.147883992350872, 11.458222874447326, 9.529314953834096, 11.234748721189295, 11.136765075414074, 9.378894559245643, 10.848941006451156, 12.877941766820157, 13.245320107085673, 16.016926096765783], [0.23904572186687872, 0.23904572186687872, 0.23904572186687872, 0.23904572186687872, 0.23904572186687872, 0.23904572186687872, 0.23904572186687872, 0.23904572186687872, 0.23904572186687872, 0.23904572186687872  …  0.23904572186687872, 0.23904572186687872, 0.23904572186687872, 0.23904572186687872, 0.23904572186687872, 0.23904572186687872, 0.23904572186687872, 0.23904572186687872, 0.23904572186687872, 0.23904572186687872], "leahy", 0.01, 100.0, 

In [15]:
# Access frequency and power arrays
freqs = ps.freq
power = ps.power

# For averaged spectrum
avg_freqs = avg_ps.freq  
avg_power = avg_ps.power

# noise reduction
println("Single spectrum noise level: ", std(ps.power[end-100:end]))
println("Averaged spectrum noise level: ", std(avg_ps.power[end-100:end]))

Single spectrum noise level: 0.038308065391872204
Averaged spectrum noise level: 1.574512782603885


In [11]:
# theoretical vs actual noise reduction
println("Number of segments used: ", avg_ps.m)
theoretical_reduction = sqrt(avg_ps.m)
actual_reduction = 0.038308065391872204 / 0.001135552971324592
println("Theoretical noise reduction[brattes method]: ", theoretical_reduction)
println("Actual noise reduction: ", actual_reduction)
println("Efficiency: ", actual_reduction / theoretical_reduction * 100, "%")

Number of segments used: 70
Theoretical noise reduction[brattes method]: 8.366600265340756
Actual noise reduction: 33.73516371251874
Efficiency: 403.2123280978188%


In [ ]:
# what I are doing here//=>
# Out of 732 total segments, only 70 high-quality segments were retained.

# The filtered 662 segments had a mean rate of ~0.0 counts/bin, indicating they were essentially empty or represented data gaps.

# In contrast, the retained 70 segments had a mean rate of ~303.8 counts/bin, reflecting periods of strong X-ray emission.

# Selection Bias (Intentional): The filtering process naturally selected only the most astrophysically meaningful segments, representing periods when the source was actually active.

# Quality Over Quantity: Rather than averaging hundreds of noisy, low-information segments, we focused on fewer, pristine and homogeneous segments, which significantly improves statistical reliability.

# Noise Reduction: By removing segments that contribute only noise (i.e., the empty or low-flux segments), the variance is greatly reduced, and the averaging becomes much more coherent.